In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from pathlib import Path
import warnings
import psycopg2
from sqlalchemy import create_engine
warnings.filterwarnings('ignore')
import sys
import pickle
from pprint import pprint

sys.path.insert(0, os.path.realpath('..\src'))
from database.database_config import DB_NAME, DB_USER, DB_PASSWORD, DB_HOST

In [2]:
match_merge = pd.read_pickle(os.path.realpath('../data/preprocessed/match_merged.pkl'))

In [3]:
team_dimension_extract = match_merge[['home']]

In [4]:
team_dimension_extract['home'].sort_values().unique()

array(['Alavés', 'Athletic Club', 'Atlético Madrid', 'Barcelona', 'Betis',
       'Celta Vigo', 'Cádiz', 'Eibar', 'Elche', 'Espanyol', 'Getafe',
       'Girona', 'Granada', 'Huesca', 'La Coruña', 'Las Palmas',
       'Leganés', 'Levante', 'Mallorca', 'Málaga', 'Osasuna',
       'Rayo Vallecano', 'Real Madrid', 'Real Sociedad', 'Sevilla',
       'Valencia', 'Valladolid', 'Villarreal'], dtype=object)

In [6]:
team_dimension_extract

,home
0,Leganés
1,Valencia
2,Celta Vigo
3,Girona
4,Sevilla
...,...
1895,Sevilla
1896,Real Sociedad
1897,Granada
1898,Elche


In [7]:
team_dim = [i for i in team_dimension_extract['home'].sort_values().unique()]

In [8]:
team_dim

['Alavés',
 'Athletic Club',
 'Atlético Madrid',
 'Barcelona',
 'Betis',
 'Celta Vigo',
 'Cádiz',
 'Eibar',
 'Elche',
 'Espanyol',
 'Getafe',
 'Girona',
 'Granada',
 'Huesca',
 'La Coruña',
 'Las Palmas',
 'Leganés',
 'Levante',
 'Mallorca',
 'Málaga',
 'Osasuna',
 'Rayo Vallecano',
 'Real Madrid',
 'Real Sociedad',
 'Sevilla',
 'Valencia',
 'Valladolid',
 'Villarreal']

In [9]:
teams_dimension = pd.DataFrame(team_dim, columns = ['team_name'])

In [10]:
teams_dimension

,team_name
0,Alavés
1,Athletic Club
2,Atlético Madrid
3,Barcelona
4,Betis
5,Celta Vigo
6,Cádiz
7,Eibar
8,Elche
9,Espanyol


In [11]:
db = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}")
conn = db.connect()

In [12]:
teams_dimension.to_sql(
        "teams", con=conn, schema="laliga_wh", if_exists="replace", index=False
    )

28

In [13]:
conn.execute(
        """ALTER TABLE laliga_wh.teams 
                        ADD COLUMN team_id SERIAL PRIMARY KEY;"""
    )
conn.close()